In [47]:
import pandas as pd
import numpy as np
import psycopg2
import datetime
import matplotlib.pyplot as plt
import io

conn = psycopg2.connect(
    host = 'localhost',
    dbname = 'testedb',
    user = 'csk',
    password = 'csk123',
    port = 5432
)
cur = conn.cursor()

#cur.execute("CREATE TABLE IF NOT EXISTS confirmed (id serial PRIMARY KEY, cidade varchar, data date, acumulado_confirmados integer, novos_confirmados integer);")
#cur.execute("CREATE TABLE IF NOT EXISTS death (id serial PRIMARY KEY, cidade varchar, data date, acumulado_mortes integer, novos_mortes integer);")
#conn.commit()

pd.set_option('display.notebook_repr_html', True)
def _repr_latex_(self):
    return "\centering{%s}" % self.to_latex()
pd.DataFrame._repr_latex_ = _repr_latex_

df_completo = pd.read_csv("covid19_casos_brasil.csv")
df_completo.tail()

,city,city_ibge_code,date,epidemiological_week,estimated_population_2019,is_last,is_repeated,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_available_deaths,order_for_place,place_type,state,new_confirmed,new_deaths
353908,NaN,43.0,2020-07-12,29,11377239.0,True,False,39240,344.89914,2020-07-12,0.0245,962,125,state,RS,520,19
353909,NaN,42.0,2020-07-12,29,7164788.0,True,False,43031,600.59000,2020-07-12,0.0115,497,123,state,SC,1005,12
353910,NaN,28.0,2020-07-12,29,2298696.0,True,False,37631,1637.05858,2020-07-12,0.0261,984,121,state,SE,1585,30
353911,NaN,35.0,2020-07-12,29,45919049.0,True,False,371997,810.11477,2020-07-12,0.0480,17848,139,state,SP,5107,146
353912,NaN,17.0,2020-07-12,29,1572866.0,False,True,15132,962.06543,2020-07-11,0.0169,255,117,state,TO,0,0


In [45]:
cols = ['city', 'date', 'last_available_confirmed', 'last_available_deaths']
cidades = ['Joinville', 'Blumenau', 'Itajaí', 'Criciúma', 'Florianópolis']
frames = list()
for c in cidades:
    frames.append(df_completo[df_completo['city'] == c])
filtro_cidades = (df_completo['city'] == 'Joinville') | (df_completo['city'] == 'Blumenau') | (df_completo['city'] == 'Itajaí') | (df_completo['city'] == 'Criciúma') | (df_completo['city'] == 'Florianópolis')
df = df_completo[filtro_cidades]
df.tail()[cols]

,city,city_ibge_code,date,epidemiological_week,estimated_population_2019,is_last,is_repeated,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_available_deaths,order_for_place,place_type,state,new_confirmed,new_deaths
162,Joinville,4209102.0,2020-03-13,11,590466.0,False,False,1,0.16936,2020-03-13,0.0000,0,1,city,SC,1,0
205,Joinville,4209102.0,2020-03-14,11,590466.0,False,False,1,0.16936,2020-03-14,0.0000,0,2,city,SC,0,0
254,Joinville,4209102.0,2020-03-15,12,590466.0,False,False,1,0.16936,2020-03-15,0.0000,0,3,city,SC,0,0
308,Joinville,4209102.0,2020-03-16,12,590466.0,False,False,1,0.16936,2020-03-16,0.0000,0,4,city,SC,0,0
378,Joinville,4209102.0,2020-03-17,12,590466.0,False,False,1,0.16936,2020-03-17,0.0000,0,5,city,SC,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331370,Florianópolis,4205407.0,2020-07-08,28,500973.0,False,False,1856,370.47905,2020-07-08,0.0108,20,119,city,SC,75,0
336732,Florianópolis,4205407.0,2020-07-09,28,500973.0,False,False,1905,380.26001,2020-07-09,0.0115,22,120,city,SC,49,2
342099,Florianópolis,4205407.0,2020-07-10,28,500973.0,False,False,1980,395.23088,2020-07-10,0.0116,23,121,city,SC,75,1
347469,Florianópolis,4205407.0,2020-07-11,28,500973.0,False,False,2041,407.40719,2020-07-11,0.0127,26,122,city,SC,61,3


In [41]:
for index, row in df.iterrows():
    cur.execute("""
        INSERT INTO confirmed (cidade, data, acumulado_confirmados, novos_confirmados)
        VALUES (%(a)s, %(b)s, %(c)s, %(d)s);
        """,{'a': row['city'],
             'b': row['date'],
             'c': row['last_available_confirmed'],
             'd': row['new_confirmed']
            })
conn.commit()
cur.close()
conn.close()

In [48]:
for index, row in df.iterrows():
    cur.execute("""
        INSERT INTO death
            (cidade, data, acumulado_mortes, novos_mortes)
        VALUES
            (%(a)s, %(b)s, %(c)s, %(d)s);
        """,{'a': row['city'],
             'b': row['date'],
             'c': row['last_available_deaths'],
             'd': row['new_deaths']
            })
conn.commit()
cur.close()
conn.close()